In [7]:
import psycopg2
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point,Polygon
from datetime import date

In [8]:
db_con = {
    'host':'localhost', 
    'database':'pop',
    'user':'postgres', 
    'password':'root'
}

In [9]:
query = '''
SELECT 
    opr.opr_fim_plantio as fim_plantio,
    opr.opr_fim_colheita as fim_colheita,
    glb.glb_id as id,
    glb.glb_poligono as gleba
FROM opr_operacao AS opr
inner join 
	glb_gleba as glb
on glb.glb_opr = opr.opr_id
and glb.glb_id in (1633,100,27)
'''

In [10]:
with psycopg2.connect(**db_con) as conn:
    df = gpd.read_postgis(query,conn, geom_col='gleba')

/home/pedro/.local/lib/python3.10/site-packages/geopandas/io/sql.py:170: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [11]:
df['fim_plantio'] = pd.to_datetime(df['fim_plantio'], format='%Y-%m-%d').astype(str)
df['fim_colheita'] = pd.to_datetime(df['fim_colheita'], format='%Y-%m-%d').astype(str)
df['gleba'] = df['gleba'].apply(lambda geom: ','.join([f'[{x},{y}]' for x, y in geom.exterior.coords]))
df.dtypes

/tmp/ipykernel_4326/2256651839.py:3: UserWarning: Geometry column does not contain geometry.
  df['gleba'] = df['gleba'].apply(lambda geom: ','.join([f'[{x},{y}]' for x, y in geom.exterior.coords]))


fim_plantio     object
fim_colheita    object
id               int64
gleba           object
dtype: object

In [13]:
for row in df.iterrows():
    print(row[1]['id'])
    url = f'https://indices-svsux4mgsa-uc.a.run.app/time_series?dt_antes={row[1]["fim_plantio"]}&dt_depois={row[1]["fim_colheita"]}&safra=1&filter=%22whittaker%22&geom=[[{row[1]["gleba"]}]]'
    print(url)

27
https://indices-svsux4mgsa-uc.a.run.app/time_series?dt_antes=2018-10-20&dt_depois=2019-02-20&safra=1&filter=%22whittaker%22&geom=[[[-52.571149,-23.467578],[-52.566832,-23.460926],[-52.566836,-23.460606],[-52.566873,-23.460442],[-52.566643,-23.460208],[-52.566501,-23.460056],[-52.566339,-23.460092],[-52.56588,-23.459437],[-52.565422,-23.458627],[-52.565582,-23.458198],[-52.565861,-23.457928],[-52.565818,-23.457586],[-52.566169,-23.45701],[-52.573432,-23.462384],[-52.572141,-23.465653],[-52.571625,-23.467043],[-52.571149,-23.467578]]]
100
https://indices-svsux4mgsa-uc.a.run.app/time_series?dt_antes=2018-10-27&dt_depois=2019-02-28&safra=1&filter=%22whittaker%22&geom=[[[-52.618858,-23.501058],[-52.620499,-23.502401],[-52.620982,-23.501889],[-52.621143,-23.501801],[-52.621368,-23.501574],[-52.621561,-23.501348],[-52.621679,-23.501141],[-52.621744,-23.500895],[-52.619555,-23.499346],[-52.617227,-23.497737],[-52.616519,-23.498327],[-52.61609,-23.498721],[-52.615693,-23.499065],[-52.616519,